## <7-2 질의응답 모델 학습하기>  

질의응답 모델의 데이터 전처리와 학습 과정을 실습해 보겠습니다.

### 질문에 답하는 모델 만들기  
---
<**1단계**> **코랩 노트북 초기화하기**       


이전 실습과 마찬가지로 코랩에서 `[내 드라이브에 복사]`를 진행합니다. 그리고 코랩의 메뉴에서 [런타임 $\rightarrow$ `런타임 유형 변경`]을 클릭하고 `[GPU]나 [TPU] 둘 중 하나를 선택`(GPU 권장)해 하드웨어 가속을 사용합니다.

<**2단계**> **각종 설정하기**     

다음 코드를 차례로 실행해 TPU 관련 라이브러리와 그 외에 의존성 있는 패키지를 설치하고 자신의 구글 드라이브를 코랩 노트북과 연결합니다. 다만 1단계에서 GPU를 선택했다면 코드 7-1은 생략합니다.

In [ ]:
#TPU 관련 패키지 설치
#!pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl

In [1]:
#의존성 패키지 설치
!pip install ratsnlp

     |████████████████████████████████| 42 kB 761 kB/s 
     |████████████████████████████████| 806 kB 8.4 MB/s 
     |████████████████████████████████| 2.8 MB 33.8 MB/s 
     |████████████████████████████████| 57 kB 4.8 MB/s 
     |████████████████████████████████| 636 kB 47.6 MB/s 
     |████████████████████████████████| 136 kB 44.5 MB/s 
     |████████████████████████████████| 829 kB 43.1 MB/s 
     |████████████████████████████████| 398 kB 31.9 MB/s 
     |████████████████████████████████| 895 kB 31.2 MB/s 
     |████████████████████████████████| 3.3 MB 31.0 MB/s 
     |████████████████████████████████| 77 kB 4.4 MB/s 
     |████████████████████████████████| 1.1 MB 43.7 MB/s 
     |████████████████████████████████| 96 kB 5.9 MB/s 
     |████████████████████████████████| 144 kB 47.5 MB/s 
     |████████████████████████████████| 271 kB 46.9 MB/s 
     |████████████████████████████████| 94 kB 3.3 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha

In [2]:
#구글 드라이브와 연결
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


이번 실습에서는 `kcbert-base` 모델을 `KorQuAD 1.0` 데이터로 파인튜닝해 볼 예정입니다.  
다음 코드를 실행하면 관련 설정을 할 수 있습니다.

In [7]:
#모델 환경 설정
import torch
from ratsnlp.nlpbook.qa import QATrainArguments
args = QATrainArguments(
    pretrained_model_name="beomi/kcbert-base",
    downstream_corpus_name="korquad-v1",
    downstream_model_dir="/gdrive/My Drive/nlpbook/checkpoint-qa",
    max_seq_length=128,
    max_query_length=32,
    doc_stride=64,
    batch_size=32 if torch.cuda.is_available() else 4,
    learning_rate=5e-5,
    epochs=3,
    tpu_cores=0 if torch.cuda.is_available() else 8,
    seed=7,
)

참고로 `QATrainArguments`의 각 인자는 4장 문서 분류 실습에서 설명한 것과 같지만, 특별히 이번 실습에서 `max_seq_length`, `max_query_length`, `doc_stride`의 역할과 의미는 다음과 같습니다.  



- **max_seq_length**: 입력 문장 최대 길이(질문과 지문 모두 포함)   
- **max_query_length**: 질문 최대 길이   
- **doc_stride**: 지문에서 몇 개 토큰을 슬라이딩해가면서 데이터를 늘릴지 결정. cf) '5단계 데이터 전처리 하기' 참고

이어서 다음 코드를 차례로 실행해 랜덤 시드를 설정하고, 각종 기록을 출력하는 로거를 설정합니다.

In [8]:
#랜덤 시드 고정
from ratsnlp import nlpbook
nlpbook.set_seed(args)

set seed: 7


In [9]:
#로거 설정
nlpbook.set_logger(args)

INFO:ratsnlp:Training/evaluation parameters QATrainArguments(pretrained_model_name='beomi/kcbert-base', downstream_corpus_name='korquad-v1', downstream_corpus_root_dir='/content/Korpora', downstream_model_dir='/gdrive/My Drive/nlpbook/checkpoint-qa', max_seq_length=128, doc_stride=64, max_query_length=32, threads=4, cpu_workers=2, save_top_k=1, monitor='min val_loss', seed=7, overwrite_cache=False, force_download=False, test_mode=False, learning_rate=5e-05, epochs=3, batch_size=32, fp16=False, tpu_cores=0, tqdm_enabled=True)


<**3단계**> **말뭉치 내려받기**    

다음 코드를 실행하면 이 실습에서 사용할 KorQuAD 1.0 데이터셋을 내려받습니다.  
데이터셋을 내려받을 때는 `nlpbook`에 포함된 `download_downstream_dataset()` 함수를 사용합니다. 이 함수에 코드 '모델 환경 설정'에서 선언한 `args`를 전달하면 `downstream_corpus_name(korquad-v1)`에 해당하는 말뭉치를 내려받습니다.

In [10]:
#말뭉치 다운로드
nlpbook.download_downstream_dataset(args)

Downloading: 38.5MB [00:00, 70.0MB/s]
Downloading: 3.88MB [00:00, 50.1MB/s]                  


<**4단계**> **토크나이저 준비하기**    

다음 코드를 실행해 `kcbert-base` 모델이 사용하는 토크나이저를 선언합니다.

In [11]:
#토크나이저 준비
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False,
)

Downloading:   0%|          | 0.00/250k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/619 [00:00<?, ?B/s]